In [1]:
# libraries needed to start spark session
import pyspark
from pyspark.sql import SparkSession

# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits

# for everything else
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
from progressbar import ProgressBar
import pickle

In [2]:
# start new spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("drones1") \
    .getOrCreate()

# to view events/jobs in current spark session:
spark

In [4]:
# read in as binary
path = "/home/developer/notebooks/Images"
df1 = spark.read.format("binaryFile")\
        .option("pathGlobFilter", "*.png")\
        .option("recursiveFileLookup", "true")\
        .load(path)

df1.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



In [5]:
df2 = df1.rdd.map(lambda x: x["content"])
n_images = df2.count()
content = df2.take(n_images)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, 8a191f3ece30, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:503)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:638)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:62

In [6]:
# Using the VGG16 model from keras
# https://keras.io/api/applications/vgg/#vgg16-function
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

553467904/553467096 [==============================] - 3s 0us/step ETA: 


In [7]:
# https://github.com/keras-team/keras/issues/11684
from PIL import Image
import io

data = {}

for i in range(n_images):
    img = Image.open(io.BytesIO(content[i]))
    img1 = img.convert('RGB')
    img1 = img1.resize((224,224))
    
    img2 = np.array(img1)
    reshaped_img = img2.reshape(1,224,224,3)
    imgx = preprocess_input(reshaped_img)
    features = model.predict(imgx, use_multiprocessing=True)
    data[i] = features

with open('data.pickle','wb') as file:
    pickle.dump(data,file)

NameError: name 'n_images' is not defined

In [ ]:
with open('data.pickle','rb') as file:
    data = pickle.load(file)

In [ ]:
feat = np.array(list(data.values())).reshape(-1,4096)
pca = PCA(n_components = 36, random_state = 22)
pca.fit(feat)
x = pca.transform(feat)

In [ ]:
print("Components before PCA:", feat.shape[1])
print("Components after PCA:", pca.n_components)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=22)
kmeans.fit(x)

In [ ]:
kmeans.labels_

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(data.keys(),kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
# Create folders for each cluster in host
for i in range(len(unique_labels)):
    # makeadir() evaluates your condition
    out_path = 'D:\Clusters\{}'.format(i)
    if not os.path.exists(out_path):
        os.mkdir(out_path)

In [1]:
pbar2 = ProgressBar()

NameError: name 'ProgressBar' is not defined

In [ ]:
# Put all images of each cluster into thier respective folders on host for examination
source_path = r"D:\training"
# Create array from images in source path
source_list = []
for images in os.scandir(source_path):
        source_list.append(images)
# For loor copies images over into the clustered folders
for i in pbar2(range(len(kmeans.labels_))):
    folder = kmeans.labels_[i]
    out_path = r'D:\Clusters\{}'.format(folder)
    image_select = source_list[i]
    shutil.copy(image_select, out_path)

In [ ]:
sse = []
list_k = list(range(2, 20))

In [ ]:
for k in list_k:
    km = KMeans(n_clusters=k, random_state=22)
    km.fit(x)
    
    sse.append(km.inertia_)

In [ ]:
# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');